# VTA 测试

In [ ]:
import tvm
from tvm import relay
from tvm.relay import Call
from tvm.relay.op.annotation import compiler_begin, compiler_end
from tvm.relay.testing import run_opt_pass
from tvm.relay import op, transform
from tvm.relay import ExprMutator, ExprVisitor
# from vta.top.graphpack import ExprPack
from vta.top.graphpack import (
    run_opt_pass, 
    _to_shape,
    _pack_batch_channel,
    _unpack_batch_channel,
    # _channel_const_match, # 新增
    _const_shape_match,
    _weight_shape_match,
    # _weight_shape_match_transpose, # 新增
    _pack_weight,
    _pack_weight_conv2d_transpose,
    # _pack_const, # 被修改
    _get_tensor_shape,
    _get_tensor_type,
)

In [ ]:
from torch import nn
import torch

class Model(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.conv = nn.Conv2d(3, 16, 3, 1, 1, bias=False)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

pt_model = Model().eval().float()
ishape = (1, 3, 4, 4)
input_shapes = [("data", ishape)]
# script_module = torch.jit.script(pt_model)
# mod, params = relay.frontend.from_pytorch(script_module, input_shapes)
idata = torch.randn(ishape).type(torch.float32)
traced_model = torch.jit.trace(pt_model, idata)
# traced_model 翻译为 TVM 前端模型
mod, params = relay.frontend.from_pytorch(traced_model, input_shapes, use_parser_friendly_name=True)
# 量化
with tvm.transform.PassContext(opt_level=3):
    with relay.quantize.qconfig(skip_conv_layers=[]):
        mod = relay.quantize.quantize(mod, params)
mod.show()

In [ ]:
# import vta
# env = vta.get_env()
# bitpack_start = op.op.get("annotation.bitpack_start")
# bitpack_end = op.op.get("annotation.bitpack_end")
# # anf = run_opt_pass(expr, transform.ToANormalForm())
# operator_current_idx = 0
# bfactor = env.BATCH
# cfactor = env.BLOCK_OUT
# weight_bits = env.WGT_WIDTH
# # expr = run_opt_pass(anf, transform.ToGraphNormalForm())

In [ ]:
expr = mod["main"]
assert isinstance(expr, relay.Function)
expr = run_opt_pass(expr, transform.InferType())  
# packer = ExprPack(bfactor, cfactor, weight_bits)
# expr = packer.visit(expr)
# expr = run_opt_pass(expr, transform.InferType())

In [34]:
ExprMutator.visit_call??

Signature: ExprMutator.visit_call(self, call)
Docstring: <no docstring>
Source:   
    def visit_call(self, call):
        new_fn = self.visit(call.op)
        new_args = [self.visit(arg) for arg in call.args]
        if new_fn == call.op and new_args == list(call.args):
            return call
        return Call(new_fn, new_args, call.attrs, call.type_args, call.span)
File:      /media/pc/data/lxw/ai/tvm/xinetzone/__pypackages__/3.10/lib/tvm/relay/expr_functor.py
Type:      function

In [ ]:
dev_annotater = ExprAnnotateDev()
ff = dev_annotater.visit(expr)

In [ ]:
run_mod = tvm.IRModule.from_expr(ff)
run_mod = relay.transform.MergeCompilerRegions()(run_mod)

In [ ]:
print(run_mod)